VDSR for DEM processing - Tuner
====================================
**Using the [Keras Tuner](https://www.tensorflow.org/tutorials/keras/keras_tuner) to pick the optimal set of hyperparameters for a Very Deep Super Resolution (VDSR) CNN.**

More about machine learning techniques for improved resolution of Digital Elevation Models:  [Super-resolution](https://opendem.info/opendemsearcher.html)

The original VDSR implementation was done by George Seif [GitHub](https://github.com/GeorgeSeif/VDSR-Keras).

This Notebook is running on Google Colab with data storage on Google Drive. Of course this could be adapted to local resources.


In [1]:
import os
# connect with google drive  
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


Get rasterio for GeoTiff processing

In [2]:
!pip install rasterio

In [3]:
import os
import tensorflow as tf
import numpy as np
import rasterio
from tensorflow.python.data.experimental import AUTOTUNE
from rasterio.plot import reshape_as_image, reshape_as_raster, show
import re

# adapt your paths
HR_PATH_TRAIN = "/content/drive/My Drive/vdsr4dem/hr_train"
LR_PATH_TRAIN = "/content/drive/My Drive/vdsr4dem/lr_train"
HR_PATH_VAL = "/content/drive/My Drive/sr_images/hr_valid"
LR_PATH_VAL = "/content/drive/My Drive/sr_images/lr_valid"


# maximal and minimal values of the whole trainings dataset
ds_max = 3150
ds_min = 170

# use human sorting to avoid problems with the order of the files for training and validation
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]

def _images_dataset(image_files):    
    listAll = []
    for img in image_files:
        with rasterio.open(img) as src:
            array = src.read(
                out_shape=(
                    src.count,
                    int(src.height),
                    int(src.width)
                )
            )
            array = reshape_as_image(array)            
            lr = (array - ds_min) / (ds_max - ds_min)
            listAll.append(lr)        
    ds = tf.data.Dataset.from_tensor_slices(listAll)
    #print(ds)
    return ds

def _get_dataset(path, num_images):
    file_names = [f"{path}/{x}" for x in os.listdir(path) if x.find(".tif") != -1]
    if num_images is not None:
        file_names = file_names[:num_images]
    print("--->", len(file_names))
    file_names.sort(key=natural_keys)
    print(file_names)
    ds = _images_dataset(file_names)    
    return ds


def dataset(batch_size=16, repeat_count=None, random_transform=True, use_set="train", num_images=None):
    print(num_images)
    if use_set == "train":
        print('LR_PATH_TRAIN')
        lr_ds = _get_dataset(LR_PATH_TRAIN, num_images)
        print('HR_PATH_TRAIN')
        hr_ds = _get_dataset(HR_PATH_TRAIN, num_images)
    elif use_set == "val":
        print('LR_PATH_VAL')
        lr_ds = _get_dataset(LR_PATH_VAL, num_images)
        print('HR_PATH_VAL')
        hr_ds = _get_dataset(HR_PATH_VAL, num_images)
    else:
        return None
    
    ds = tf.data.Dataset.zip((lr_ds, hr_ds))
    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    return ds

if __name__ == "__main__":
    # To avoid problems with the order of the files always set num_images to the real number of images in the folder!
    train_ds = dataset(batch_size=16, use_set="train", num_images=101)
    val_ds = dataset(batch_size=1, use_set="val", num_images=31)
    print("done all")

Get the Keras Tuner

In [4]:
!pip install keras-tuner

# Step 1
Pick the optimal hyperparaters:


*   filter kernel size
*   filters
*   learning rate   



In [5]:
from kerastuner import HyperModel
from keras.models import Sequential, Model
from keras import Sequential
from keras.layers import Conv2D
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Input, add
from keras import optimizers
from keras import losses
from keras.optimizers import SGD, Adam

class RegressionHyperModel(HyperModel):
    def __init__(self, input_shape):
        self.input_shape = input_shape        

    def build(self, hp):
        model = keras.Sequential()

        filter = hp.Choice('filter', values = [4,8,16,32,64]);
        kernel = hp.Choice('kernel', values = [5,9,15,21]);


        model.add(
            Conv2D(
                filters= filter,
                kernel_size= kernel,
                padding='same',
                kernel_initializer='he_normal',
                activation='relu',
                input_shape=(300, 300, 1)
            )
        )
        model.add(
            Conv2D(
                filters= filter,
                kernel_size= kernel,
                padding='same',
                kernel_initializer='he_normal',
                activation='relu',
                input_shape=model.output_shape
            )
        )
        model.add(
            Conv2D(
                filters= filter,
                kernel_size= kernel,
                padding='same',
                kernel_initializer='he_normal',
                activation='relu',
                input_shape=model.output_shape
            )
        )
        model.add(
            Conv2D(
                filters= filter,
                kernel_size= kernel,
                padding='same',
                kernel_initializer='he_normal',
                activation='relu',
                input_shape=model.output_shape
            )
        )
        model.add(
            Conv2D(
                filters= filter,
                kernel_size= kernel,
                padding='same',
                kernel_initializer='he_normal',
                activation='relu',
                input_shape=model.output_shape
            )
        )
        model.add(
            Conv2D(
                filters= filter,
                kernel_size= kernel,
                padding='same',
                kernel_initializer='he_normal',
                activation='relu',
                input_shape=model.output_shape
            )
        )
        model.add(
            Conv2D(
                filters= filter,
                kernel_size= kernel,
                padding='same',
                kernel_initializer='he_normal',
                activation='relu',
                input_shape=model.output_shape
            )
        )
        model.add(
            Conv2D(
                filters= filter,
                kernel_size= kernel,
                padding='same',
                kernel_initializer='he_normal',
                activation='relu',
                input_shape=model.output_shape
            )
        )
        model.add(
            Conv2D(
                filters= filter,
                kernel_size= kernel,
                padding='same',
                kernel_initializer='he_normal',
                activation='relu',
                input_shape=model.output_shape
            )
        )
        model.add(
            Conv2D(
                filters= filter,
                kernel_size= kernel,
                padding='same',
                kernel_initializer='he_normal',
                activation='relu',
                input_shape=model.output_shape
            )
        )
        model.add(
            Conv2D(
                filters= filter,
                kernel_size= kernel,
                padding='same',
                kernel_initializer='he_normal',
                activation='relu',
                input_shape=model.output_shape
            )
        )
        model.add(
            Conv2D(
                filters= filter,
                kernel_size= kernel,
                padding='same',
                kernel_initializer='he_normal',
                activation='relu',
                input_shape=model.output_shape
            )
        )
        model.add(
            Conv2D(
                filters= filter,
                kernel_size= kernel,
                padding='same',
                kernel_initializer='he_normal',
                activation='relu',
                input_shape=model.output_shape
            )
        )
        model.add(
            Conv2D(
                filters= filter,
                kernel_size= kernel,
                padding='same',
                kernel_initializer='he_normal',
                activation='relu',
                input_shape=model.output_shape
            )
        )
        model.add(
            Conv2D(
                filters= filter,
                kernel_size= kernel,
                padding='same',
                kernel_initializer='he_normal',
                activation='relu',
                input_shape=model.output_shape
            )
        )
        model.add(
            Conv2D(
                filters= filter,
                kernel_size= kernel,
                padding='same',
                kernel_initializer='he_normal',
                activation='relu',
                input_shape=model.output_shape
            )
        )
        model.add(
            Conv2D(
                filters= filter,
                kernel_size= kernel,
                padding='same',
                kernel_initializer='he_normal',
                activation='relu',
                input_shape=model.output_shape
            )
        )
        model.add(
            Conv2D(
                filters= filter,
                kernel_size= kernel,
                padding='same',
                kernel_initializer='he_normal',
                activation='relu',
                input_shape=model.output_shape
            )
        )
        model.add(
            Conv2D(
                filters= filter,
                kernel_size= kernel,
                padding='same',
                kernel_initializer='he_normal',
                activation='relu',
                input_shape=model.output_shape
            )
        )
        model.add(
            Conv2D(
                filters= filter,
                kernel_size= kernel,
                activation='relu',
                padding='same',
                kernel_initializer='he_normal',             
                input_shape=model.output_shape
            )
        )
        model.compile(
            optimizer=keras.optimizers.Adam(
                hp.Choice('learning_rate',
                      values=[0.001, 0.0001, 0.00001, 0.000001])),
            loss='mse',
            metrics=['mse']
        )
        return model

In [6]:
from tensorflow import keras
from kerastuner.tuners import RandomSearch

hypermodel = RegressionHyperModel(input_shape=(300, 300, 1))

tuner_rs = RandomSearch(
 hypermodel,
 objective='mse',
 seed=42,
 max_trials=100,
 # adapt path
 directory='/content/drive/My Drive/tunerdirectory12',
 executions_per_trial=5)

INFO:tensorflow:Reloading Oracle from existing project /content/drive/My Drive/tunerdirectory12/untitled_project/oracle.json


In [7]:
tuner_rs.search_space_summary()

Search space summary
Default search space size: 3
filter (Choice)
{'default': 4, 'conditions': [], 'values': [4, 8, 16, 32, 64], 'ordered': True}
kernel (Choice)
{'default': 5, 'conditions': [], 'values': [5, 9, 15, 21], 'ordered': True}
learning_rate (Choice)
{'default': 0.001, 'conditions': [], 'values': [0.001, 0.0001, 1e-05, 1e-06], 'ordered': True}


In [ ]:
N_EPOCH_SEARCH = 50

tuner_rs.search(train_ds, validation_data=val_ds, epochs=N_EPOCH_SEARCH)

tuner_rs.results_summary()

# Step 2
Use a loop (1-30) to pick the optimum number of CNN layers.

In [12]:
from kerastuner import HyperModel
from keras.models import Sequential, Model
from keras import Sequential
from keras.layers import Conv2D
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Input, add
from keras import optimizers
from keras import losses
from keras.optimizers import SGD, Adam

class RegressionHyperModel(HyperModel):
    def __init__(self, input_shape):
        self.input_shape = input_shape        

    def build(self, hp):
        model = keras.Sequential()
        
        model.add(
            Conv2D(
                filters=32,
                kernel_size=9,
                activation='relu',
                padding='same',
                kernel_initializer='he_normal',             
                input_shape=(300, 300, 1)
            )
        )

        for i in range(hp.Int('n_layers', 1, 30)):  # adding variation of layers.
            model.add(
                Conv2D(
                    filters=32,
                    kernel_size=9,
                    activation='relu',
                    padding='same',
                    kernel_initializer='he_normal',             
                    input_shape=(300, 300, 1)
                )
            )

        
        model.add(
            Conv2D(
                filters=1,
                kernel_size=9,
                activation='relu',
                padding='same',
                kernel_initializer='he_normal',             
                input_shape=model.output_shape
            )
        )
        adam = Adam(lr=0.0001)
        model.compile(
            adam,
            loss='mse',
            metrics=['mse']
        )
        return model

In [14]:
from tensorflow import keras
from kerastuner.tuners import RandomSearch

hypermodel = RegressionHyperModel(input_shape=(300, 300, 1))

tuner_rs = RandomSearch(
 hypermodel,
 objective='mse',
 seed=42,
 max_trials=100,
 # adapt path
 directory='/content/drive/My Drive/tunerdirectory14',
 executions_per_trial=5)

In [15]:
tuner_rs.search_space_summary()

Search space summary
Default search space size: 1
n_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 30, 'step': 1, 'sampling': None}


In [ ]:
N_EPOCH_SEARCH = 50

tuner_rs.search(train_ds, validation_data=val_ds, epochs=N_EPOCH_SEARCH)

tuner_rs.results_summary()